In [1]:
import os
import sys
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import pandas as pd
import torchaudio
import codes.utils
from codes.preprocess_data import *

In [2]:
main = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [3]:
FOLDER_ANNOTATIONS = "datos/"
ANNOTATIONS_FILE = ["train_set.txt","test_set.txt"]
N_FOLDS = 4
MBE_DIR_TR = "features/"+ ANNOTATIONS_FILE[0].split(".")[0]+"/"
MBE_DIR_TE = "features/"+ ANNOTATIONS_FILE[1].split(".")[0]+"/"
SAMPLE_RATE = 44100
LEN_SEC = 300
LEN_SAMPLES = LEN_SEC*SAMPLE_RATE
__class_labels = {
'perro'    : 0,
'rana'     : 1,
'lluvia'   : 2,
'motor'    : 3,
'ave'      : 4
}
N_FFT = 2048
HOP = int(N_FFT/2)
N_MELS = 40

In [4]:
df_train = pd.read_csv(FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[0],sep=" ",
                    names=["filepath","start","end","class_name"])
audio_filenames_train = np.unique(df_train.filepath)

In [5]:
df_test = pd.read_csv(FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[1],sep=" ",
                    names=["filepath","start","end","class_name"])
audio_filenames_test = np.unique(df_test.filepath)

In [6]:
for i,file in enumerate(audio_filenames_train):
    mbe,label = preprocess_data(file,main,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[0],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    namembe = "{}mbe{}.pt".format(MBE_DIR_TR,numberfile)
    namembe = os.path.join(main,namembe)
    std = torch.std(mbe,2).view(mbe.shape[0],mbe.shape[1],1)
    mean = torch.mean(mbe,2).view(mbe.shape[0],mbe.shape[1],1)
    mbe = (mbe-mean)/std
    torch.save({"data":mbe,"label":label}, namembe)

/home/vpoblete/Escritorio/tesis/notebooks/sednet_pytorch/codes/preprocess_data.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor = torch.tensor(label,device=device)


In [7]:
for i,file in enumerate(audio_filenames_test):
    mbe,label = preprocess_data(file,main,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[1],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    namembe = "{}mbe{}.pt".format(MBE_DIR_TE,numberfile)
    namembe = os.path.join(main,namembe)
    std = torch.std(mbe,2).view(mbe.shape[0],mbe.shape[1],1)
    mean = torch.mean(mbe,2).view(mbe.shape[0],mbe.shape[1],1)
    mbe = (mbe-mean)/std
    torch.save({"data":mbe,"label":label}, namembe)